In [2]:

from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
firefox_profile = webdriver.FirefoxProfile()
firefox_profile.set_preference("browser.privatebrowsing.autostart", True)


#Storing previous dates from current date
def get_prev_dates(year,month,day):
    from datetime import date, timedelta
    dates=[]
    sdate = date(int(year), int(month), int(day))   # start date
    edate= date(2018, 10 ,14)
    #edate = date.today()   # end date
    delta = edate - sdate       # as timedelta
    for i in range(delta.days):
        d = sdate + timedelta(days=i)
        dates.append(d.strftime('%Y-%m-%d'))

    return dates[:-2]

dates=get_prev_dates(2018,1,1)   
#Create function to get only numbers/decimal values 
import re
def remove_char(s):
    new = ''
    for letter in s:
        if not(letter.isalpha()):
            new+=letter
    new = re.sub('[%/°]', '', new)
    new = re.sub(' ', '', new)
    return new 

#get the webpage
try:
    fox = webdriver.Firefox(executable_path=GeckoDriverManager().install(),firefox_profile=firefox_profile)
    fox.get("https://www.worldweatheronline.com/lang/en-in/bangalore-weather-history/karnataka/in.aspx")
    fox.maximize_window() 
    fox.implicitly_wait(3)
except Exception as e:
            print('Error opening browser through selenium')

#create dataframe
import pandas as pd
df=pd.DataFrame()
data={}
#looping through each date/page
for date in dates:
    try:
        fox.find_element_by_css_selector('#ctl00_MainContentHolder_txtPastDate').send_keys(date)
        fox.find_element_by_xpath('//*[@id="ctl00_MainContentHolder_butShowPastWeather"]').click()
        WebDriverWait(fox, 3).until(EC.presence_of_element_located((By.XPATH, '/html/body/form/div[4]/main/div[6]/div/div[2]/div[11]')))
        time.sleep(2)
        i=0
        if int(date.split("-")[0])==2020:
            i=120
        if int(date.split("-")[0])==2019:
            i=110
        if int(date.split("-")[0])==2018:
            i=100
        if int(date.split("-")[0])==2017:
            i=90
        if int(date.split("-")[0])==2016:
            i=80
        if int(date.split("-")[0])==2015:
            i=70    
        if int(date.split("-")[0])==2014:
            i=60  
        if int(date.split("-")[0])==2013:
            i=50  
        if int(date.split("-")[0])==2012:
            i=40  
        if int(date.split("-")[0])==2011:
            i=30   
        if int(date.split("-")[0])==2010:
            i=20     
        if int(date.split("-")[0])==2009:
            i=10    
        data['Date']=date
        data['Max_Temp(°c)']=float(remove_char(fox.find_element_by_xpath('/html/body/form/div[4]/main/div[6]/div/div[2]/div['+str(i+3)+']').text))
        data['Min_Temp(°c)']=float(remove_char(fox.find_element_by_xpath('/html/body/form/div[4]/main/div[6]/div/div[2]/div['+str(i+4)+']').text))
        data['Wind_Speed(Km/h)']=float(remove_char(fox.find_element_by_xpath('/html/body/form/div[4]/main/div[6]/div/div[2]/div['+str(i+5)+']').text))
        data['Wind_Direction']=fox.find_element_by_xpath('/html/body/form/div[4]/main/div[6]/div/div[2]/div['+str(i+5)+']').text.split("\n")[-1]
        data['Rain(mm)']=float(remove_char(fox.find_element_by_xpath('/html/body/form/div[4]/main/div[6]/div/div[2]/div['+str(i+6)+']').text))
        data['Humidity(%)']=float(remove_char(fox.find_element_by_xpath('/html/body/form/div[4]/main/div[6]/div/div[2]/div['+str(i+7)+']').text))
        data['Cloud_Density(%)']=float(remove_char(fox.find_element_by_xpath('/html/body/form/div[4]/main/div[6]/div/div[2]/div['+str(i+8)+']').text))
        data['Atm_Pressure(mb)']=float(remove_char(fox.find_element_by_xpath('/html/body/form/div[4]/main/div[6]/div/div[2]/div['+str(i+9)+']').text))
        df=df.append(data,ignore_index=True)
        df.index+=1
    except Exception as e:
            print('Error: Data for particular date not availabel', date)
            continue


df.to_csv('bangalore_rain_data2.csv',index=False)
print('Scraping Completed..!!!', 'Search for bangalore_rain2.csv')
fox.quit()

[WDM] - Looking for [geckodriver v0.26.0 macos] driver in cache 
[WDM] - File found in cache by path [/Users/abhisekkumar/.wdm/drivers/geckodriver/v0.26.0/macos/geckodriver]


Scraping Completed..!!! Search for bangalore_rain2.csv


/Users/abhisekkumar/Library/Python/3.6/lib/python/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  This is separate from the ipykernel package so we can avoid doing imports until
